### Программистская часть

#### Задача 1. 

Напишите черновик игры в жанре RPG. Идеологически: игрок выбирает, будет ли он играть за волшебника или за бойца, а потом выбранным героем сражается с монстрами, набирая очки опыта. Что должно быть технически:

- классы волшебника и бойца (можно создать отдельный класс Player и наследоваться от него, но необязательно)
- класс монстра (хотя бы один)
- класс оружия (тут тоже фантазию не ограничиваю - можно создать абстрактный класс и наследоваться от него, можно сделать классы для меча и для посоха с варьирующими атрибутами)
- класс Игра, в котором будут все необходимые методы
- все это должно быть разложено по отдельным скриптам .py в папке, класс игры импортируется в файл main.py, и его методы вызываются там. 

In [ ]:
    # your code here

#### Задача 2. 

Дан текст, каждая строка которого является полным или относительным путём к некоторому файлу.
Напишите регулярное выражение, которое захватывает:
1. директорию, в которой лежит файл;
2. только имя файла (без расширения);
3. только расширение;
При этом:
- нужны только файлы, у которых расширение не .bat и не .txt.
- пути могут быть как в Unix, так и в Windows формате (https://ru.wikipedia.org/wiki/Путь_к_файлу).
- расширение, если оно есть, начинается с точки. Файлы могут быть без расширения вовсе (в этом случае на месте расширения должно стоять None или "")
- скрытые файлы могут начинаться с точки (например, .bashrc - и это не расширение)
- относительный путь может содержать только название файла, в этом случае вместо директории выведите None или ""
- в остальных случаях директория должна заканчиаться на разделитель директорий. Наприемр, в Unix-системах - "/" - это путь к корневой директории.
Требуется получить список из кортежей, каждый кортеж содержит извлечённые данные.
Используйте флаг VERBOSE, чтобы не запутаться.
(Расширение в целом может содержать всё, что угодно, но разделителей директорий не может быть в именах файлах и расширениях. https://en.wikipedia.org/wiki/List_of_filename_extensions )

In [ ]:
# your code here

#### Задача 3. 

Жизнь.
Напишите игру "Жизнь".
Что это такое - читайте в википедии и здесь: http://www.michurin.net/online-tools/life-game.html
Вообще говоря, это не игра в привычном понимании этого слова, а процесс.
В простейшем виде достаточно раз в 0.1 секунды выводить на экран обновлённое поле. Для рамочек можно использовать специальные символы для рисования рамочек (найдите в таблице unicode). Пробел - пустая клетка, живая клетка может быть обозначена, например, символом '+'. Начальное поле генерируется случайным образом, 
вероятность появления жизни в клетке при начальной генерации - должна быть настраиваемым параметром. Размеры поля вводит пользователь при запуске программы. Также должна быть возможность в качестве начальной популяции использовать R-pentomino (http://www.conwaylife.com/wiki/R-pentomino)

In [ ]:
from copy import deepcopy
from IPython.display import clear_output
from random import randint

In [ ]:
# вообще-то я не в восторге от идеи делать специальный класс под клетку
# потому что и атрибутов, и методов тут не то чтобы много, зато начинаются проблемы со ссылками (а было бы так здорово просто хранить 2 больших списка-поля...)
# но сделать просто один большой класс Game_of_Life было бы как-то почти бессмысленно (тогда уже проще всё на одних функциях писать), а задача вроде про ООП
class Cell:
    def __init__(self, is_alive, coords, field):
        self.is_alive = is_alive
        self.x, self.y = coords
        self.field = field
        self.neighbors_coords_list = [(self.x + i, self.y + j) for i in range(-1, 2) for j in range(-1, 2) if not i == j == 0 and
                          0 <= (self.x + i) < self.field.width and 0 <= (self.y + j) < self.field.heigth]
    def __repr__(self):
        return str(self.is_alive)

    def get_neighbors_sum(self):
        return sum(self.field.data[coord[1]][coord[0]].is_alive for coord in self.neighbors_coords_list)

In [ ]:
class Game_of_Life:
    def __init__(self, width, heigth):
        if (heigth <= 0 or width <= 0
            or type(heigth) != int or type(width) != int):
            raise ValueError('Ширина и высота должны быть целыми положительными числами')
        self.width = width
        self.heigth = heigth
        self.data = []
        self.ask_first_intention()
    
    def __str__(self):
        image = '╔' + '═' * (self.width + 2) + '╗\n'
        for y in range(self.heigth):
            image += '║ ' + ''.join(['♥' if self.data[y][x].is_alive else '-' for x in range(self.width)]) + ' ║\n'
        image += '╚' + '═' * (self.width + 2) + '╝'
        return image
    
    def __iter__(self):
        for y in range(self.heigth):
            for x in range(self.width):
                yield self.data[y][x].is_alive
    # общение
    def ask_first_intention(self):
        intent = input('''Итак поле готово, нажмите:
1 — если хотите зародить жизнь на поле рандомно (надо будет задать вероятность)

2 — если хотите поставить на поле R-pentomino

3 — если хотите сделать какую-то конкретную клетку живой (вы сможете сделать это и позже\n''')
        if intent == '1':
            try:
                self.set_random_field(int(input('Введите желаемую вероятность зарождения жизни в процентах — целое число от 0 до 100\n')))
                self.ask_next_intention()
            except:
                print('Вероятность зарождения жизни должна быть указана в процентах\n(т. е. быть целым числом от 0 до 100)')
                self.ask_first_intention()
        elif intent == '2':
            self.set_empty_field()
            coord = input('Через пробел введите координаты верхней правой (!) клетки вашего R-pentomino\n').split()
            try:
                self.set_r_pentomino(int(coord[0]), int(coord[1]))
                self.ask_next_intention()
            except:
                print('Что-то не так с форматом, либо ваш R-pentomino не поместился в поле')
                self.ask_first_intention()
        elif intent == '3':
            coord = input('Через пробел (!) введите координаты клетки, которую хотите сделать живой\n').split()
            try:
                self.set_cell(int(coord[0]), int(coord[1]), 1)
                self.ask_next_intention()
            except:
                print('Что-то не так с форматом либо ваша клетка не поместилась в поле')
                self.ask_first_intention()
        else:
            print('Я не понял, давайте ещё раз...')
            self.ask_first_intention()
    
    def ask_next_intention(self):
        intent = input('''Теперь можно нажать
1 — чтобы запустить жизнь

2 — чтобы запустить жизнь, но смотреть на неё пошагово

3 — чтобы сделать конкретную клетку живой

4 — чтобы убить конкретную клетку

5 — чтобы просто полюбоваться полем :)\n''')
        # можно было, конечно придумать специальную функцию, а не писать if-else
        # но так как разные пути требуют разной цепочки вводов в дальнейшем
        # не сильно стало бы красивее от такой функции
        if intent == '1':
                self.draw()
        elif intent == '2':
            self.draw_by_steps()
        elif intent == '3':
            coord = input('Через пробел (!) введите координаты клетки, которую хотите сделать живой\n').split()
            try:
                self.set_cell(int(coord[0]), int(coord[1]), 1)
            except:
                print('Что-то не так с форматом либо ваша клетка не поместилась в поле')
            self.ask_next_intention()
        elif intent == '4':
            coord = input('Через пробел (!) введите координаты клетки, которую хотите убить\n').split()
            try:
                self.set_cell(int(coord[0]), int(coord[1]), 0)
            except:
                print('Что-то не так с форматом либо ваша клетка не поместилась в поле')
            self.ask_next_intention()
        elif intent == '5':
            print(self)
            self.ask_next_intention()
        else:
            print('Я не понял давайте ещё раз...')
            self.ask_next_intention()

    # настройка поля
    def set_empty_field(self):
        self.data = []
        for y in range(self.heigth):
            line = []
            for x in range(self.width):
                line.append(Cell(0, (x, y), self))
            self.data.append(line)

    def set_random_field(self, livebility):
        self.data = []
        if type(livebility) != int or not (0 <= livebility < 100):
            raise ValueError('Вероятность зарождения жизни указана не в процентах')
        for y in range(self.heigth):
            line = []
            for x in range(self.width):
                line.append(Cell(int(randint(0, 100) <= livebility), (x, y), self))
            self.data.append(line)

    def set_r_pentomino(self, x, y):
        cells = [(x, y), (x - 1, y), (x - 1, y + 1),
                (x - 2, y + 1), (x - 1, y + 2)]
        for coord in cells:
            if not (0 <= coord[0] < self.width) or not (0 <= coord[1] < self.heigth):
                raise ValueError('R-pentomino не поместился в поле :(')
            self.data[coord[1]][coord[0]].is_alive = 1
        print(self)
    
    def set_cell(self, x, y, is_alive):
        if not self.data: self.set_empty_field()
        if type(is_alive) != int or not (is_alive in [0, 1]):
            raise ValueError('Клетка либо жива, либо мертва')        
        if type(x) != int or not (0 <= x < self.width):
            raise ValueError('x не поместился в поле :(')
        if type(y) != int or not (0 <= y < self.heigth):
            raise ValueError('y не поместился в поле :(')
        self.data[y][x].is_alive = is_alive
    
    # движ    
    def upd(self):
        # вот тут вскрывается неоптимальность класса Cell
        # я воспользовалась модулем, если так нельзя, то пусть я просто ещё раз цикл в цикле добавила :(
        prev_state = deepcopy(self.data)
        # я задумалась о том, чтобы сделать что-то с map, как-то так чтобы только уже живые и их соседи проверялись (ну типа оптимизация)
        # но что-то так и не придумала ничего
        # поэтому довольно медленно оно работает к сожалению
        for y in range(self.heigth):
            for x in range(self.width):
                if (prev_state[y][x].get_neighbors_sum() == 3) or (prev_state[y][x].get_neighbors_sum() + prev_state[y][x].is_alive == 3):
                    self.data[y][x].is_alive = 1
                else:
                    self.data[y][x].is_alive = 0

    def draw(self):
        while sum(self):
            clear_output(wait = True)
            print(self)
            self.upd()
        clear_output(wait = True)
        print(self)
    
    def draw_by_steps(self):
        input_text = ':)'
        while input_text != '0':
            clear_output(wait = True)
            print(self)
            self.upd()
            input_text = input('Введите 0, если хотите остановиться\nВведите что-нибудь другое, если хотите продолжать\n')

In [ ]:
f = Game_of_Life(40, 20)

### Лингвистическая часть

Для выполнения этих заданий выберите два любых достаточно длинных текста (.txt) на русском и на любом другом (для которого есть парсеры) языке; если возьмете текст и его перевод, будет отлично.

#### Задача 4. 

Просмотрите оба выбранных текста. Удостоверьтесь, что тексты чистые, если же в них есть какой-то мусор: хештеги, затесавшиеся при OCR символы и подобное, почистите с помощью регулярных выражений. 

Проведите первичный статистический анализ: разбейте тексты на предложения и на токены, посчитайте относительное количество того и другого, сопоставьте. Если ваши тексты параллельные, какой длиннее? В каком тексте средняя длина предложения больше? Почему? В каком тексте выше лексическое разнообразие? 

Таким образом, вам необходимо узнать следующие вещи:

- количество предложений (относительное и абсолютное)
- количество токенов (относительное и абсолютное)
- средняя длина предложения (среднее количество слов в предложении)
- соотношение "уникальные токены / все токены"
- (опционально) соотношение знаков пунктуации и слов

In [ ]:
# https://mojsrpski.org/reading/parallel — источник
# тексты — "Мост на Дрине" (1 глава) параллельно на русском и сербском
with open('most_na_drine_rus.txt', 'r', encoding='utf-8') as file:
    rus_text = file.read() # UDPipe с моделью синтагруса хуже разбиравет "ёлочки", чем кавычки
with open('most_na_drine_srb.txt', 'r', encoding='utf-8') as file:
    srb_text = file.read()

In [ ]:
# количество предложений можно, конечно, было попытаться посчитать регуляркой
# что-то типа:
import re
pattern = r'\s?(\(?(?:[»\"].+?[«\"]|.)+?(?:(?<!\s\w)\.|\?|!){1,3}\)?)\s?'
# sents = re.findall(pattern, rus_text.replace('\n', ' '), flags=(re.MULTILINE))
# sents = re.findall(pattern, srb_text.replace('\n', ' '), flags=(re.MULTILINE))
# print(sents)

In [ ]:
# но я сразу возьму UDPipe (запрета на UDPipe до 5-ого задания я не нашла (: )
from conllu import parse
with open('processed_rus.conllu', 'r', encoding='utf-8') as rus_conllu:
    parsed_rus = parse(rus_conllu.read())

with open('processed_srb.conllu', 'r', encoding='utf-8') as srb_conllu:
    parsed_srb = parse(srb_conllu.read())

In [ ]:
# # потому что регулярка не идеальна, и nltk.sent_tokenize() тоже так себе работает (если раскомментировать, будет видно, почему)
# import nltk
# # nltk.download('punkt')
# sentences = nltk.sent_tokenize(srb_text)
# for i in range(len(parsed_srb)):
#     if parsed_srb[i].metadata['text'] != sents[i]:
#         print('PREV:\n' + parsed_srb[i - 1].metadata['text'] + '\n\n' + sentences[i - 1] + '\n\n' + sents[i - 1] + '\n\n\n')
#         print('CURR:\n' + parsed_srb[i].metadata['text'] + "\n\n" + sentences[i]+ '\n\n' + sents[i] + '\n\n\n')
#         print('NEXT:\n' + parsed_srb[i+1].metadata['text'] + "\n\n" + sentences[i+1]+ '\n\n' + sents[i+1] + '\n\n\n')

In [ ]:
# чтобы всякие относительные вещи считать
def percent(numerator, divisor):
    return round(numerator * 100 / divisor, 2)

количество предложений

In [ ]:
# можно было взять больше одной главы, но там довольно точный перевод, поэтому большой разницы там бы всё равно не было
# (я не взяла, потому что в сербском тексте вручную расставлены переносы, ручками убирать их я не хочу,
# а настолько хорошие регулярки писать не умею)
print(f'{len(parsed_srb)} предложений — в оригинальном тексте на сербском')
print(f'{len(parsed_rus)} предложений — в переводе на русский')
print(f'{percent(len(parsed_rus), len(parsed_srb))}% — количества предложений в оригинале количество предложений в переводе на русский составляет')

количество токенов

In [ ]:
count_rus_tokens = sum(len(sent) for sent in parsed_rus)
count_srb_tokens = sum(len(sent) for sent in parsed_srb)
print(f'{count_srb_tokens} токенов — в оригинальном тексте на сербском')
print(f'{count_rus_tokens} токенов — в переводе на русский')
print(f'{percent(count_rus_tokens, count_srb_tokens)}% от количества токенов на сербском составляет количество токенов на русском')

длина предложений

In [ ]:
def len_sent(sent):
    return sum(map(lambda token: token.get('upos') != 'PUNCT', sent))

def mean_len(parsed_text):
    return round(sum(len_sent(sent) for sent in parsed_text) / len(parsed_text), 3)

print(f'{mean_len(parsed_srb)} — средняя длина сербского предложения')
print(f'{mean_len(parsed_rus)} — средняя длина русского предложения')
print(f'{percent(mean_len(parsed_rus), mean_len(parsed_srb))}% от средней длины сербского предложенияя составляет средняя длина русского')

уникальные токены

In [ ]:
def count_uniq_tokens(parsed_text):
    res = set()
    for sent in parsed_text:
        res.update(set(token.get('form') for token in sent))
    return res

# здорово, какая большая разница!
print(f'{percent(len(count_uniq_tokens(parsed_srb)), sum(len(sent) for sent in parsed_srb))}% процент уникальных токенов в сербском тексте')
print(f'{percent(len(count_uniq_tokens(parsed_rus)), sum(len(sent) for sent in parsed_rus))}% процент уникальных токенов в русском тексте')

знаки препинания на слово

In [ ]:
def count_puncts_words(parsed_text):
    punct_counter = 0
    word_counter = 0
    for sent in parsed_text:
        punct_in_sent = sum(map(lambda token: token.get('upos') == 'PUNCT', sent))
        punct_counter += punct_in_sent
        word_counter +=  (len(sent) - punct_in_sent)
    return punct_counter, word_counter
punct_rus, word_rus = count_puncts_words(parsed_rus)
punct_srb, word_srb = count_puncts_words(parsed_srb)

print(f'В сербском тексте:\n\t{word_srb}\tслов ({percent(word_srb,(word_srb + punct_srb))}% токенов)\n\t{punct_srb}\tзнаков препинания ({percent(punct_srb, (word_srb + punct_srb))}% токенов)\n\t{round(word_srb / punct_srb, 2)}\tслов на 1 знак препинания')
print(f'В русском тексте:\n\t{word_rus}\tслов ({percent(word_rus,(word_rus + punct_rus))}% токенов)\n\t{punct_rus}\tзнаков препинания ({percent(punct_rus, (word_rus + punct_rus))}% токенов)\n\t{round(word_rus / punct_rus, 2)}\tслов на 1 знак препинания')

#### Задача 5. 

Сделайте морфосинтаксические разборы ваших текстов в формате UD, запишите .conllu-файлы. 

In [ ]:
# можно скачать (https://lindat.mff.cuni.cz/repository/xmlui/handle/11234/1-4923#) модельки
# через ufal.udpipe к ним обращаться локально
# нашла для русского отдельно какую-то модель (не очень хорошую): https://github.com/ancatmara/data-science-nlp/raw/master/data/russian-ud-2.0-170801.udpipe, для сербского не нашла
# в полной версии это полгигабайта, и менять я их не собираюсь, зачем тогда скачивать

In [ ]:
# чтобы показать, что я умею:
from ufal.udpipe import Model
from ufal.udpipe import Pipeline
import wget

wget.download('https://github.com/ancatmara/data-science-nlp/raw/master/data/russian-ud-2.0-170801.udpipe')
model = Model.load('russian-ud-2.0-170801.udpipe')
pipeline = Pipeline(model, 'tokenize', Pipeline.DEFAULT, Pipeline.DEFAULT, Pipeline.DEFAULT)
ex_text = 'Я нашла для русского отдельно какую-то модель (не очень хорошую)'
print(pipeline.process(ex_text))

In [ ]:
# можно использовать API, но так нельзя парсить крупные тексты
# пришлось бы отправлять по 1-2 предложения
import requests
import json

def get_conllu_with_api(text, model):
    url = 'http://lindat.mff.cuni.cz/services/udpipe/api/process'
    params = {
        'tokenizer': '', 'tagger': '', 'parser': '', 
        'model': model,
        'input': 'generic_tokenizer',
        'data': text}
    response = requests.get(url, params)
    return response.json()['result']

ex_text_rus = 'можно использовать API, но так нельзя парсить крупные тексты'
ex_text_srb = 'moguće je koristiti API, ali tako ne možete strugati velike textove'
print(get_conllu_with_api(ex_text_rus, 'russian-syntagrus-ud-2.10-220711'))
print(get_conllu_with_api(ex_text_srb, 'serbian-set-ud-2.10-220711'))

In [ ]:
# поэтому просто ручками вставила текст сюда: http://lindat.mff.cuni.cz/services/udpipe/run.php
# использовала 'russian-syntagrus-ud-2.10-220711' для русского b 'serbian-set-ud-2.10-220711' для сербского
# и сохранила output-файлы

In [ ]:
# если нужно было написать парсер conllu-файлов самому, у меня уже есть, правда, довольно плохой, старый код, который это делает
from conllu import parse
with open('processed_rus.conllu', 'r', encoding='utf-8') as rus_conllu:
    parsed_rus = parse(rus_conllu.read())

with open('processed_srb.conllu', 'r', encoding='utf-8') as srb_conllu:
    parsed_srb = parse(srb_conllu.read())

#### Задача 6. 

Посчитайте статистику по частям речи, сопоставьте: можно напечатать две таблички с процентами по частям речи. 

In [ ]:
def tokens_param_stat(parsed_text, param, *upos_exceptions):
    res = {}
    for sentence in parsed_text:
        for token in sentence:
            if not token.get('upos') in upos_exceptions:
                if token.get(param) not in res:
                    res[token.get(param)] = 0
                res[token.get(param)] += 1
    return res

def pos_stat_in_table(pos_dict):
    all_tokens_sum = sum(pos_dict[pos] for pos in pos_dict)
    res = 'Часть речи\tКоличество\tВ процентах\n\n'
    for pos in sorted(pos_dict.keys()):
        res += f'{pos}\t\t{pos_dict[pos]}\t\t{percent(pos_dict[pos], all_tokens_sum)}%\n'
    return res

In [ ]:
counted_pos_rus = tokens_param_stat(parsed_rus, 'upos', 'PUNCT')
print('Текст на русском:\n\n' + pos_stat_in_table(counted_pos_rus))

In [ ]:
# есть загадочная строка:
# 32	tarih	tar	X	Ncmsn	Foreign=Yes	30	appos	_	TokenRange=4702:470
# не знаю, что это и верно ли оно распознанно (вряд ли), и такое слово всего одно, поэтому просто убрала его из подсчётов
counted_pos_srb = tokens_param_stat(parsed_srb, 'upos', 'PUNCT', 'X')
print('Текст на сербском:\n\n' + pos_stat_in_table(counted_pos_srb))

#### Задача 7. 

Посчитайте, какое соотношение токенов по частям речи имеет совпадающие со словоформой леммы (т.е., в скольких случаях токены с частью речи VERB, например, имели словарную форму: и сам токен, и лемма одинаковые). Что вы можете сказать о выбранных вами языках на основании этих данных? Ожидаются две таблички с процентами несовпадающих по лемме и токену слов для каждой части речи. 

In [ ]:
def dict_forms_counter(parsed_text, *exceptions):
    res = {}
    for sentence in parsed_text:
        for token in sentence:
            if not token.get('upos') in exceptions:
                if token.get('upos') not in res.keys(): 
                    res[token.get('upos')] = 0
                res[token.get('upos')] += int(token.get('form') == token.get('lemma'))
    return res

def dict_forms_table(dict_forms_pos_dict, pos_dict):
    res = 'Часть речи\tФорм, совпадающих с леммами\tВсего форм\tПроцент совпадения\n\n'
    for pos in sorted(pos_dict.keys()):
        res += f'{pos}\t\t{dict_forms_pos_dict[pos]}\t\t\t\t{pos_dict[pos]}\t\t{round(dict_forms_pos_dict[pos] * 100 / pos_dict[pos], 2)}%\n'
    return res

In [ ]:
counted_dict_forms_rus = dict_forms_counter(parsed_rus, 'PUNCT')
print(dict_forms_table(counted_dict_forms_rus, counted_pos_rus))

In [ ]:
counted_dict_forms_srb = dict_forms_counter(parsed_srb, 'PUNCT', 'X')
print(dict_forms_table(counted_dict_forms_srb, counted_pos_srb))

##### *Что я могу сказать о выбранных мной языках на основании этих данных:*

*В этих текстах русский и сербский примерно одинаково синтетические: "более изменяемые" части речи "компенсируют" менее изменяемые, тут дела по-разному у русского и сербского.*

*Были бы это не 2 славянских языка, а, например, русский и корейский, разница была бы больше, полагаю.*

#### Задача 8. 

Посчитайте медианную длину предложения для ваших текстов (медиана - это если взять все длины всех ваших предложений, упорядочить их от маленького к большому и выбрать то число, которое оказалось посередине, а если чисел четное количество, то взять среднее арифметическое двух чисел посередине. Например, если у вас пять предложений длинами 1, 2, 6, 7, 8, то медиана - 6, а если шесть предложений длинами 1, 1, 7, 9, 10, 11, то медиана - (7 + 9) / 2 = 8). Возьмите любые два предложения (одно русское и второе на другом языке) и постройте для них деревья зависимостей. Изучите связи зависимостей (deprel) и вершины: согласны ли вы с разбором?

In [ ]:
def get_median(nums):
    nums.sort()
    middle_index = len(nums) // 2
    if len(nums) % 2: return sorted(nums)[middle_index]
    return round((nums[middle_index - 1] + nums[middle_index]) / 2, 2)

# уже было, но повторю
def len_sent(sent):
    return sum(map(lambda token: token.get('upos') != 'PUNCT', sent))

In [ ]:
get_median([len_sent(sent) for sent in parsed_srb])
print(f'{get_median([len_sent(sent) for sent in parsed_srb])} — средняя длина сербского предложения')
print(f'{get_median([len_sent(sent) for sent in parsed_rus])} — средняя длина русского предложения')

деревья

In [ ]:
from treelib import Node, Tree
def add_one(token, sent, tree):
    if token.get('head') == 0 and not token.get('id') in tree.nodes:
        tree.create_node(str(token.get('id')) + ': ' + token.get('upos') + ' ' + token.get('form'), token.get('id'))
    elif not token.get('id') in tree.nodes and token.get('upos') != 'PUNCT':
        if not token.get('head') in tree.nodes:
            add_one(sent[token.get('head') - 1], sent, tree)
        tree.create_node(str(token.get('id')) + ': ' + token.get('upos') + ' ' + token.get('form').lower() + ' (' + token.get('deprel') + ')',
                         token.get('id'), parent=token.get('head'))
def draw_tree(sent):
    tree = Tree()
    for token in sent:
        add_one(token, sent, tree)
    tree.show()

In [ ]:
draw_tree(parsed_rus[3])
draw_tree(parsed_srb[7])

##### *Согласна ли я с разбором*
*Не очень:*
- *в нём участвуют знаки препинания*
- *вместо привычной синтаксической вершины предложной группы — вершина подчинённой именной, получается* **PP \[ NP \[ AdjP \] \]** *вместо* **NP \[ \[ PP \] \[ AdjP \] \]** *(а в "друг к другу" даже не подчинённой, но реципрокальные местоимения — это вообще не спортивно, если мы про синтаксис)*
- *подчинение придаточных: по идее должно быть 16 → 18 → 21 или 16 → 21 → 18 (это спорный момент), но никак не 21 → 16 + 21 → 18*
- *не то чтобы я знала сербский, но то, что отрицательная частица зависит от существительного:* **(deprel:advmod) form:ne lemma:ne upos:PART** *точно неправильно*

*Я не знаю почему так устроено, зато знаю, что это не баг, а фича, и оно в принципе так работает*

#### Задача 9. 

Посчитайте частотные списки токенов для каждой категории связей зависимостей (т.е., нужно выделить все токены в тексте, которые получали, например, ярлык amod, и посчитать их частоты). Выведите по первые три самых частотных токена для каждой категории (punct можно не выводить). 

In [ ]:
def deprel_stat_in_table(deprel_dict):
    all_tokens_sum = sum(deprel_dict[kind] for kind in deprel_dict)
    res = 'Тип зависимости\tКоличество\tВ процентах\n\n'
    for kind in sorted(deprel_dict.keys()):
        kind_str = kind.ljust(8, ' ')
        res += f'{kind_str}\t{deprel_dict[kind]}\t\t{percent(deprel_dict[kind], all_tokens_sum)}%\n'
    return res

In [ ]:
deprel_stat_rus = tokens_param_stat(parsed_rus, 'deprel', 'PUNCT')
print('В тексте на русском:\n\n' + deprel_stat_in_table(deprel_stat_rus))

deprel_stat_srb = tokens_param_stat(parsed_srb, 'deprel', 'PUNCT')
print('В тексте на сербском:\n\n' + deprel_stat_in_table(deprel_stat_srb))

#### Задача 10. 

Некоторые предлоги в русском языке могут управлять разными падежами (например, "я еду в Лондон" vs "я живу в Лондоне"). Давайте проанализируем эти предлоги и их падежи. Необходимо:

- составить список таких предлогов (РГ-80 вам в помощь)
- взять достаточно большой текст (можно большое художественное произведение)
- сделать морфоразбор этого текста (лучше не pymorphy)
- Посчитать, как часто и какие падежи встречаются у слова, идущего после предлога.

Примечания: во-первых, имейте в виду, что иногда после предлога могут идти самые неожиданные вещи: "я что, должен ехать на, черт побери, северный полюс?". Во-вторых, неплохо бы учитывать отсутствие пунктуации (конечно, в норме, как нам кажется, предлог обязательно требует зависимое, но! "да иди ты на!") Эти штуки можно отсеять, если просто учитывать только заранее определенные падежи, а не считать все, какие встретились (так и None можно огрести).

Если будете использовать RNNMorph, возможно, понадобится регулярное выражение и немного терпения.

*составить список таких предлогов*

In [ ]:
prepositions = {'между': {'твор', 'род'},
                'меж': {'твор', 'род'},
                'с': {'род', 'вин', 'твор'},
                'по': {'дат', 'вин', 'местн', 'пр'},
                'в': {'вин', 'местн', 'пр'},
                'за': {'вин', 'твор'},
                'на': {'вин', 'местн', 'пр'},
                'о': {'вин', 'местн', 'пр'},
                'под': {'вин', 'твор'}}

*взять достаточно большой текст*

In [ ]:
import re

with open('piramidi.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    
text_justified_spaces = re.sub(r'[\s"\(\)]+', ' ', text) #кавычки и скобки убираем, потому что потом может идти слово в нужном падеже

with open('text_to_lem.txt', 'w', encoding='utf-8') as f:
    f.write(text_justified_spaces)

*сделать морфоразбор этого текста*

In [ ]:
!mystem -n -c -i -d --format json text_to_lem.txt lem_res.json

In [ ]:
import json

with open('lem_res.json', 'r', encoding='utf-8') as f:
    lem_dict = [json.loads(line) for line in f.readlines() if line != '{"text":" "}\n']
    lemmatized = list(filter(lambda x: x != '{"text":" "}\n', f.readlines()))

*Посчитать, как часто и какие падежи встречаются у слова, идущего после предлога*

Если просто смотреть следующее слово

In [ ]:
# варианты падежей, которые в принципе подчиняются предлогам (т. е. кроме им, парт, зват)
def get_possible_cases(token_dict):
    res = set()
    for var in token_dict['analysis']:
        var_case = re.findall('[=,](род|вин|дат|твор|местн|пр)(?:,|$)', var['gr'])
        # всё-таки руководствуемся РГ-80, и считаем, что местного не существует, но при желании replace можно просто убрать и статка чуть поменяется
        if var_case: res.add(var_case[0].replace('местн', 'пр'))
    return res

In [ ]:
res = dict()
for i in range(len(lem_dict)):
    if ('analysis' in lem_dict[i] and len(lem_dict[i]['analysis'])
        and lem_dict[i]['analysis'][0]['lex'] in prepositions):
        # если следуюющий токен без анализа
        if not 'analysis' in lem_dict[i + 1] or not len(lem_dict[i + 1]['analysis']):
                continue
        next_word_case = get_possible_cases(lem_dict[i + 1])
        if next_word_case:
            next_word_case_str = ' || '.join(next_word_case)
            if not next_word_case_str in res: res[next_word_case_str] = 0
            res[next_word_case_str] += 1

for case in sorted(res.keys()):
    print(case.ljust(33) + str(res[case]))

Если заморачиваться — пытаться учитывать синкретизм, родительный падеж влево (*'в его глазах'*), числа и всякое такое

In [ ]:
# получить часть речи
def get_pos(token_dict):
    if not 'analysis' in token_dict and token_dict['text'].isdigit():
        return 'DIGIT'
    if not 'analysis' in token_dict or not len(token_dict['analysis']):
        return 'no_pos'
    res = token_dict['analysis'][0]['gr'][:token_dict['analysis'][0]['gr'].find('=')]
    if res.count(','): return res[:res.find(',')]
    return res

In [ ]:
import re
# варианты падежей в пересечении с теми, которые допускает предлог
def get_possible_cases(token_dict, prepos_cases):
    res = set()
    for var in token_dict['analysis']:
        var_case = re.findall(r'[=,](род|вин|дат|твор|местн|пр)(?:,|$)', var['gr'])
        # всё-таки руководствуемся РГ-80, и считаем, что местного не существует, но при желании replace можно просто убрать и статка чуть поменяется
        if var_case: res.add(var_case[0].replace('местн', 'пр'))
    return res & prepos_cases

In [ ]:
# какие встречаются части речи после предлогов
pos_list = set()
for i in range(len(lem_dict)):
    if ('analysis' in lem_dict[i] and len(lem_dict[i]['analysis'])
        and lem_dict[i]['analysis'][0]['lex'] in prepositions):
        next_word_pos = get_pos(lem_dict[i + 1])
        if next_word_pos not in pos_list: pos_list.add(next_word_pos)
pos_list

In [ ]:
# те, для которых следующее слово никогда не смотрим
check_case = {'NUM', 'S', 'SPRO'}
# те, для которых следующее слово смотрим всегда
check_next = {'ADV', 'ADVPRO', 'DIGIT'}
# те, для которых следующее слово смотрим, только если падеж этого не однозначен
check_next_if_many = {'A', 'ANUM', 'APRO', 'V'}
# те, которые не войдут в анализ вообще — мусор или слишком сложно
skip = {'CONJ', 'PART', 'PR', 'no_pos'}

In [ ]:
def update_res(case_set):
    if case_set:
        res_key = ' || '.join(case_set)
        if not res_key in res: res[res_key] = 0
        res[res_key] += 1

In [ ]:
# так убого, потому что нормальное ограничение для рекурсии не придумала
# тоже не совершенный алгоритм: для примера "на совершенно неприличного вида сливу" в копилку "на" пойдёт "вин", но таких случаев ничтожно мало, вроде бы
res = dict()
for i in range(len(lem_dict)):
    if ('analysis' in lem_dict[i] and len(lem_dict[i]['analysis'])
        and lem_dict[i]['analysis'][0]['lex'] in prepositions):
        index_to_check = i + 1
        prepos_cases = prepositions[lem_dict[i]['analysis'][0]['lex']]
        next_word_pos = get_pos(lem_dict[index_to_check])
        if next_word_pos in skip:
            continue
        if next_word_pos in check_case:
            update_res(get_possible_cases(lem_dict[index_to_check], prepos_cases))
        elif next_word_pos in check_next:
            index_to_check += 1
            next_word_pos = get_pos(lem_dict[index_to_check])
            if next_word_pos in skip: continue
            possible_cases = get_possible_cases(lem_dict[index_to_check], prepos_cases)
            if len(possible_cases) == 1:
                update_res(possible_cases)
            elif (get_pos(lem_dict[index_to_check]) in check_next_if_many
                and get_pos(lem_dict[index_to_check + 1]) not in skip.union(check_next)):
                update_res(possible_cases & get_possible_cases(lem_dict[index_to_check + 1], prepos_cases))
        else:
            possible_cases = get_possible_cases(lem_dict[index_to_check], prepos_cases) 
            if len(possible_cases) == 1 or get_pos(lem_dict[index_to_check + 1]) in skip.union(check_next):
                update_res(possible_cases)
            else:
                if (possible_cases & get_possible_cases(lem_dict[index_to_check + 1], prepos_cases)):
                    update_res(possible_cases & get_possible_cases(lem_dict[index_to_check + 1], prepos_cases))
                else:
                    update_res(possible_cases)
for case in sorted(res.keys()):
    print(case.ljust(20) + str(res[case]))